In [36]:
import numpy as np 
import csv
import pandas as pd 
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [37]:
data_points = 10000

#valores aleatorios para cada q1 y q2 
q1 = np.random.uniform(-10e-6, 10e-6,data_points)
q2 = np.random.uniform(-10e-6, 10e-6,data_points)

#valores aleatorios de posicion para cada una de las cargas
r1 = np.random.uniform(-100,100,(data_points,3))
r2 = np.random.uniform(-100,100,(data_points,3))

vector_de_direccion = r1 - r2
r_de_FCoul = np.linalg.norm(r1-r2,axis=1) #magnitud de vector

In [38]:
#Creamos la base de datos de permeabilidad de material

db_permitividad = {}

with open('permitividad_de_materiales.csv',newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        material = row['Material']
        permitividad = float(row['Permitvidad relativa'])
        db_permitividad[material]=permitividad


In [39]:
print(db_permitividad)

{'Vacío': 1.0, 'Aire seco': 10059.0, 'Agua 200C': 80.1, 'Alcohol': 30.0, 'Aceite mineral': 1.7, 'Papel': 3.7, 'Poliestireno': 2.5, 'Porcelana': 5.0, 'Mica': 7.0, 'Vidrio': 5.4, 'Madera': 2.0, 'Teflón': 2.1, 'Nylon': 3.5, 'Silicio': 12.0, 'Germanio': 16.0}


In [40]:
#Volvemos a la parte de la segunda celda donde trabajamos con la Ley de Coulomb
epsilon_material = input("Material: ")
permitividad = db_permitividad.get(epsilon_material)

if permitividad is not None:
    epsilon = permitividad * 8.854e-12
    print(f"La permitividad de {epsilon_material} multiplicado por la constante de permitividad es igual a {epsilon} ")
else:
    print("No existe información del material")



No existe información del material


In [25]:
unit_vector_components = (r1-r2)/r_de_FCoul[:,np.newaxis]
F =((q1*q2)/(4*np.pi*epsilon*r_de_FCoul**2))
F_vector_components = F[:, np.newaxis] * unit_vector_components

In [28]:
dataset_de_F = np.column_stack((q1,q2,r1[:, 0], r1[:, 1], r1[:, 2], r2[:, 0], r2[:, 1], r2[:, 2],r_de_FCoul,F,F_vector_components))

In [ ]:
print(dataset_de_F[:5, :])  # Print the first 5 rows

In [30]:
df_de_FCoulomb = pd.DataFrame(dataset_de_F, columns=['q1', 'q2', 'r1x', 'r1y', 'r1z', 'r2x', 'r2y', 'r2z', 'r_de_Fcoul', 'F_magnitude', 'F_vector_x', 'F_vector_y', 'F_vector_z'])

In [31]:
df_de_FCoulomb.head()

,q1,q2,r1x,r1y,r1z,r2x,r2y,r2z,r_de_Fcoul,F_magnitude,F_vector_x,F_vector_y,F_vector_z
0,4.662699e-06,0.000001,70.121440,3.446640,-0.693134,43.250364,-39.556877,-99.126785,110.727327,0.000004,8.887340e-07,1.422298e-06,0.000003
1,3.750268e-06,0.000006,-82.846734,76.648545,2.127050,-25.222693,-95.139614,-63.763715,192.803773,0.000005,-1.630909e-06,4.862048e-06,0.000002
2,-6.744008e-07,-0.000010,90.177704,83.280759,-76.324475,-20.672426,65.293496,61.291193,177.621409,0.000002,1.152072e-06,1.869427e-07,-0.000001
3,-3.580389e-07,-0.000008,-50.517596,-71.693936,-28.347568,-65.873380,-48.393608,49.690954,82.877720,0.000004,6.931679e-07,-1.051789e-06,-0.000004
4,3.066947e-06,0.000005,-93.012180,34.792275,-91.518900,54.520389,68.621671,-66.372232,153.436117,0.000006,-6.072306e-06,-1.392387e-06,-0.000001


In [32]:
df_de_FCoulomb.tail()

,q1,q2,r1x,r1y,r1z,r2x,r2y,r2z,r_de_Fcoul,F_magnitude,F_vector_x,F_vector_y,F_vector_z
9995,-7.791404e-06,-0.000001,72.412374,-83.920594,68.230144,-96.523781,-44.370805,83.009698,174.132264,2.459240e-06,2.385856e-06,-5.585549e-07,-2.087291e-07
9996,-1.702876e-07,0.000003,90.769445,-43.592224,-99.697494,-4.071032,77.255634,86.800002,241.620025,-7.882211e-08,-3.093919e-08,3.942340e-08,6.083985e-08
9997,-6.682343e-06,-0.000004,-60.001069,81.487060,73.744500,-4.172831,-84.079413,-32.260028,204.367339,6.012829e-06,-1.642560e-06,4.871242e-06,3.118830e-06
9998,1.839673e-07,-0.000009,-42.936806,-57.586114,-82.865764,-73.963597,-33.592780,-22.563684,71.935268,-2.819090e-06,-1.215917e-06,9.402810e-07,2.363194e-06
9999,8.664119e-06,0.000004,94.722517,71.203592,-14.029551,-38.634766,68.008663,-34.994518,135.032968,1.773825e-05,1.751813e-05,4.196935e-07,2.754008e-06


In [33]:
dataset_de_resolucion_de_F = df_de_FCoulomb.to_csv('df_de_FCoulomb.csv',index= False)

In [ ]:
class PINN_Ley_de_Coulomb(nn.Module):
    def __init__(self,input_dim):
        super(PINN_Ley_de_Coulomb,self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(input_dim,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.linear(128,3)
    def forward(self,x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def custom_loss(F_Coulomb_true, F_Coulomb_pred):
    mse_loss= nn.MSELoss()(F_Coulomb_true,F_Coulomb_pred)
    
